In [ ]:
library(vegan)
library(microbiome)
library(tidyverse)
library(here)
# load data and helper functions
source("https://raw.githubusercontent.com/HenrikEckermann/in_use/master/bayesian_helper.R")
source("https://raw.githubusercontent.com/HenrikEckermann/in_use/master/mb_helper.R")
source("https://raw.githubusercontent.com/HenrikEckermann/in_use/master/reporting.R")

In [ ]:
load(here("data/data_transfer.RData"))
source(here("R/read.R"))

In [ ]:
# take over the meta variables I created in other docs
meta_new <- data_transfer[, 1:9] 

In [ ]:
# create catories for bf and childcare and specifically for ccyes vs rest
meta_new <- meta_new %>%
  mutate(
      groups = ifelse(time == "pre" & cc == "no", "noCCpre", ifelse(
          time == "pre" & cc == "yes", "CCpre", ifelse(
              time == "post" & cc == "no", "noCCpost", "CCpost"))),
      bf = ifelse(bf_ratio <= 0.25, "lowBF", ifelse(
          bf_ratio <0.75, "mediumBF", "highBF")),
      ccpost = ifelse(groups == "CCpost", 1, 0)) %>% 
  mutate(
      groups = as.factor(groups), 
      bf = as.factor(bf), 
      ccpost = as.factor(ccpost))

In [ ]:
# create new pseq object (read.R results in the object "genus" Leo created)
otu <- otu_to_df(genus, transpose = FALSE)
otu <- otu %>% 
    select(species, meta_new$sample_id) %>% 
    df_to_otu()
pseq <- phyloseq(otu, df_to_sd(meta_new), tax_table(genus))
# add diversity indeces to sample data
diversities <- 
    global(pseq, index = "all") %>% 
    select(contains("diversities")) %>% 
    rownames_to_column("sample_id")
colnames(diversities) <- gsub("diversities_", "", colnames(diversities))

sample_data(pseq) <- 
    sd_to_df(pseq) %>% 
    left_join(diversities, by = "sample_id") %>%
    df_to_sd()
meta <- sd_to_df(pseq)
# clr and relative abundance transformation to deal with compositionality of mb data
pseq.clr <- microbiome::transform(pseq, transform = "clr")
pseq.rel <- microbiome::transform(pseq, "compositional")

In [ ]:
future::availableCores()

In [ ]:
library(brms)
library(broom)
library(parallel)
options(mc.cores = 30)

In [ ]:
otus.clr <- otu_to_df(pseq.clr)
colnames(otus.clr)[which(colnames(otus.clr) == "Clostridium \\(sensu stricto\\)")] <- "Clostridium_sensu_stricto"
colnames(otus.clr) <- c("sample_id", gsub("_", "", colnames(otus.clr)[-1]))
colnames(otus.clr) <- gsub("\\.", "", colnames(otus.clr))
colnames(otus.clr) <- gsub(" ", "", colnames(otus.clr))
genus <- colnames(otus.clr)[-1]
data <- sd_to_df(pseq.clr) %>%
    left_join(otus.clr, by = "sample_id")
folder <- here("models/final_analyses")

In [1]:
# define fitting function for fixed sigma
brm_g_mi <- function(genus) {
        # delete _ for prior
        sub_prior <- gsub("_", "", genus)
        sub_prior <- gsub("\\.", "", sub_prior)
        # specify prior for mi version
        prior_n <- c(
            set_prior("normal(0, 2)", class = "b", resp = sub_prior),
            set_prior("exponential(25)", class = "sd", resp = sub_prior),
            set_prior("normal(0, 10)", class = "Intercept", resp = sub_prior), 
            set_prior("lkj(2)", class = "cor"),             
            set_prior("normal(0, 2)", class = "b", resp = "bfcounts"),
            set_prior("exponential(25)", class = "sd", resp = "bfcounts"),
            set_prior("normal(0, 10)", class = "Intercept", resp = "bfcounts")
        )
    # specify formula
    f1 <- as.formula(glue("{genus} |mi() ~ cc*time + age_d_s + mi(bf_count_s) + (1 + time + age_d_s + mi(bf_count_s)|subject_id)"))
    f2 <- as.formula(glue("bf_count_s |mi() ~ cc*time + {genus} + age_d_s + (1 + time + age_d_s + {genus}|subject_id)"))
    formula <- bf(f1) + bf(f2) + set_rescor(FALSE)
    # give individual model name for storage
    model_file <- glue("{folder}/gaussian/{genus}_full_mi")
    #fit model
    brm(
        family = gaussian(), data = data, formula = formula,
        chains = 4, warmup = 1000,
        control = control, prior = prior_n, file = model_file
        )
    
}

In [2]:
models_g <- mclapply(genus, brm_g_mi)

ERROR: Error in mclapply(genus, brm_g_mi): could not find function "mclapply"


In [3]:
# model screening/excluding gaussian
exclude_id <- c()
for (i in 1:130) {
    fit <- brm_g_mi(genus[i])
    sum_fit <- summary(fit)
    params <- rbind(sum_fit$fixed, sum_fit$random$subject_id, sum_fit$spec_pars) %>% as.data.frame()
    # extract n of divergent transitions
    n_divergent <- nuts_params(fit) %>% 
        filter(Parameter == "divergent__") %>% 
        summarise(n = sum(Value))
    # extract n of rhat > 1.1        
    n_high_rhat <- dim(filter(params, Rhat >= 1.1))[1]             
    # check if there are divergent transitions
    if (n_high_rhat > 0) {
        print(glue("{genus[i]} has {n_high_rhat} high Rhat parameter values"))
        exclude_id <- c(exclude_id, i)
    } else if (n_divergent$n > 0){
        print(glue("{genus[i]} has {n_divergent$n} divergent transitions"))
        exclude_id <- c(exclude_id, i)
    }     
}
genus_sel <- genus[-exclude_id_g]

ERROR: Error in gsub("_", "", genus): object 'genus' not found


In [4]:
calc_treatment_effect <- function(model, var_name, summarise = TRUE, stat = "mean") {
    var_name <- gsub("_", "", var_name)
    var_name <- gsub("\\.", "", var_name)
    df <- posterior_samples(model) %>%
        select(
            glue("b_{var_name}_Intercept"), 
            glue("b_{var_name}_ccyes"), 
            glue("b_{var_name}_timepost"), 
            glue("b_{var_name}_ccyes:timepost"), 
            glue("b_{var_name}_age_d_s"), 
            glue("bsp_{var_name}_mibf_count_s")
        ) %>%
        rename(
            no_pre = glue("b_{var_name}_Intercept"), 
            yes_pre = glue("b_{var_name}_ccyes"),
            no_post = glue("b_{var_name}_timepost"),
            yes_post = glue("b_{var_name}_ccyes:timepost"),
            age = glue("b_{var_name}_age_d_s"),
            bf = glue("bsp_{var_name}_mibf_count_s")
        ) %>%
        mutate(
            yes_post = no_pre + yes_pre + no_post + yes_post,
            no_post = no_pre + no_post,
            yes_pre = no_pre + yes_pre
    )    
    if (summarise) {
        df <- df %>% gather(group, value) %>%
            group_by(group) %>%
            do(data.frame(
                central = ifelse(stat == "median", median(.$value), mean(.$value)),
                lower = hpdi(.$value)[1],
                upper = hpdi(.$value)[2]
            ))
    }
    df
}

plot_effects <- function(model, var_name) {
    var_name <- gsub("_", "", var_name)
    var_name <- gsub("\\.", "", var_name)
    df_sum <- calc_treatment_effect(model, var_name)
    df <- calc_treatment_effect(model, var_name, summarise = F)
    # plot
    df %>% gather(group, value) %>%
        ggplot(aes(x = group, value)) +
            geom_jitter(alpha = 0.05, color = "darkred") +
            geom_point(data = df_sum, aes(x = group, y = central), size = 2, color = "red") +
            geom_errorbar(data = df_sum, aes(x = group, y = central,  ymin = lower, ymax = upper), color = "darkred") +
            ggtitle(var_name) +
            theme_bw()
}



In [5]:
effect_age <- tibble(genus = NA, mean = NA, lower = NA, upper = NA, prob = NA, effect = NA)
effect_bf <- tibble(genus = NA, mean = NA, lower = NA, upper = NA, prob = NA, effect = NA)
effect_yespre <- tibble(genus = NA, mean = NA, lower = NA, upper = NA, prob = NA, effect = NA)
effect_nopre <- tibble(genus = NA, mean = NA, lower = NA, upper = NA, prob = NA, effect = NA)
effect_nopost <- tibble(genus = NA, mean = NA, lower = NA, upper = NA, prob = NA, effect = NA)
for (var_name in genus_sel) {
    fit <- brm_g_mi(var_name)
    df <- calc_treatment_effect(fit, var_name, summarise = F)
    df <- df %>% 
        mutate(            
            yespost_yespre = yes_post - yes_pre,
            yespost_nopre = yes_post - no_pre,
            yespost_nopost = yes_post - no_post) %>%
        select(yespost_yespre, yespost_nopre, yespost_nopost, age, bf) %>%
        gather(effect, value) %>%
        group_by(effect) %>%
        do(data.frame(
            mean = mean(.$value),
            lower = hpdi(.$value)[1],
            upper = hpdi(.$value)[2],
            prob = mean(.$value < 0)
        ))
    effect_age <- rbind(effect_age, tibble(genus = var_name, mean = df$mean[1], lower = df$lower[1], upper = df$upper[1], prob = df$prob[1], effect = "age"))
    effect_bf <- rbind(effect_bf, tibble(genus = var_name, mean = df$mean[2], lower = df$lower[2], upper = df$upper[2], prob = df$prob[2], effect = "bf"))
    effect_yespre <- rbind(effect_yespre, tibble(genus = var_name, mean = df$mean[3], lower = df$lower[3], upper = df$upper[3], prob = df$prob[3], effect = "yespre"))
    effect_nopre <- rbind(effect_nopre, tibble(genus = var_name, mean = df$mean[4], lower = df$lower[4], upper = df$upper[4], prob = df$prob[4], effect = "nopre"))
    effect_nopost <- rbind(effect_nopost, tibble(genus = var_name, mean = df$mean[5], lower = df$lower[5], upper = df$upper[5], prob = df$prob[5], effect = "nopost"))                 
}

effect_age <- na.omit(effect_age)
effect_bf <- na.omit(effect_bf)
effect_yespre <- na.omit(effect_yespre)
effect_nopre <- na.omit(effect_nopre)
effect_nopost <- na.omit(effect_nopost)

ERROR: Error in tibble(genus = NA, mean = NA, lower = NA, upper = NA, prob = NA, : could not find function "tibble"


In [6]:
# how are effects distributed
dfs <- list(effect_age = effect_age, effect_bf = effect_bf, effect_yespre = effect_yespre, effect_nopre = effect_nopre, effect_nopost = effect_nopost)
lapply(c("effect_age", "effect_bf", "effect_yespre", "effect_nopre", "effect_nopost"), function(x){
    df <- dfs[[x]] %>% mutate(prob = report_star_nondirectional(prob))
    df1 <- filter(df, genus %in% genus_sel[1:length(genus_sel)/2])
    df2 <- filter(df, genus %in% genus_sel[length(genus_sel)/2+1:length(genus_sel)])
    p1 <- ggplot(df1, aes(genus, mean, label = prob)) +
        geom_pointrange(aes(ymin = lower, ymax = upper)) +
        theme(axis.text.x = element_text(angle = 90, hjust = 1)) +
        geom_text(nudge_y = 1) +
        ggtitle(x) +
        geom_hline(yintercept = 0, linetype = "dashed", color = "red") +
        ylim(-2, 2) +
        coord_flip() +
        theme_bw(base_size = 8)
    
    p2 <- ggplot(df2, aes(genus, mean, label = prob)) +
        geom_pointrange(aes(ymin = lower, ymax = upper)) +
        theme(axis.text.x = element_text(angle = 90, hjust = 1)) +
        geom_text(nudge_y = 1) +
        ggtitle(x) +
        geom_hline(yintercept = 0, linetype = "dashed", color = "red") +
        ylim(-2, 2) +
        coord_flip() +
        theme_bw(base_size = 8)
    list(p1, p2)
})


ERROR: Error in eval(expr, envir, enclos): object 'effect_age' not found


In [ ]:
effect_age <- effect_age %>% 
    arrange(prob) %>%
    mutate(qvalue = cummean(prob))
effect_bf <- effect_bf %>% 
    arrange(prob) %>%
    mutate(qvalue = cummean(prob))
effect_yespre <- effect_yespre %>% 
    arrange(prob) %>%
    mutate(qvalue = cummean(prob))
effect_nopre <- effect_nopre %>% 
    arrange(prob) %>%
    mutate(qvalue = cummean(prob))
effect_nopost <- effect_nopost %>% 
    arrange(prob) %>%
    mutate(qvalue = cummean(prob))
effects <- rbind(effect_age, effect_bf, effect_yespre, effect_nopre, effect_nopost)

effects %>% filter(qvalue < 0.01)

In [ ]:
# relation to effect size
effects %>%
    na.omit() %>%
    ggplot(aes(mean, prob)) +
    geom_point() +
    geom_vline(xintercept = 0, linetype = "dashed") +
    facet_wrap(~effect)